In [2]:
import numpy as np
from time import ctime
import pickle
import numpy.polynomial.polynomial
import pandas as pd
import warnings
import scipy.stats as sc
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot, plot
import plotly.figure_factory as ff
init_notebook_mode(connected=True)
from ortools.sat.python import cp_model

### Pick up functions used to generate dummy data, model, etc for sectorisation

In [15]:
def return_time_horizon_profiles_dict(random_state=None):
    
    time_horizon_profiles_dict = {
        'flat' : 
            {'y_values' : np.array(np.full((10),1)),
             'order' : 4
            },
        'beginning_heavy' : 
            {'y_values' : np.array([1.  , 1.15, 1.45, 1.15, 1.  , 0.95, 0.9 , 0.85, 0.8 , 0.75]),
             'order' : 4
            },
        'end_heavy' : 
            {'y_values' : np.array([0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  , 1.15, 1.45, 1.15, 1.  ]),
             'order' : 4
            },
        'beginning_and_end_heavy' : 
            {'y_values' : np.array([1.05, 1.35, 1.05, 0.9 , 0.65, 0.65, 0.9 , 1.05, 1.35, 1.05]),
             'order' : 4
            },
        'middle_heavy' : 
            {'y_values' : np.array([0.7 , 0.85, 1.  , 1.15, 1.3 , 1.3 , 1.15, 1.  , 0.85, 0.7 ]),
             'order' : 4
            },
        'random' : 
            {'y_values' : sc.norm.rvs(1,0.3,10,random_state=random_state).clip(min=0),
             'order' : 6
            }    
    }
    
    return time_horizon_profiles_dict

def get_time_horizon_profiles_by_time_interval(time_horizon_profile, number_time_intervals,random_state=None):
    
    """
    This function is used to return a set of weights used to vary the taskload over the time horizon, 
    using the time_horizon_profiles_dict. The base weighting in the dictionary is fitted to a 
    polynomial and used to generate a weight for each point on the time horizon. 
    """
    
    time_horizon_profiles_dict = return_time_horizon_profiles_dict()
    
    order = time_horizon_profiles_dict[time_horizon_profile]['order']
    x = np.linspace(0,number_time_intervals,10) 
    y = time_horizon_profiles_dict[time_horizon_profile]['y_values']
    ffit = np.polynomial.polynomial.Polynomial.fit(x, y, deg=order)
    time_intervals = np.arange(number_time_intervals)
    out_array = ffit(time_intervals).clip(min=0.1)
    
    return out_array

def generate_taskload_array(taskload_parameters, random_state=None):
    
    """
    Function to generate an taskload array using specified input parameters.
    """
    
    number_time_intervals = taskload_parameters['number_time_intervals']
    number_jobs = taskload_parameters['number_jobs']
    job_normal_dist_params_dict = taskload_parameters['job_normal_dist_params_dict']
    job_time_horizon_profiles_dict = taskload_parameters['job_time_horizon_profiles_dict']
    job_zero_taskload_probability_dict = taskload_parameters['job_zero_taskload_probability_dict']
    
    taskload_array = np.zeros((number_jobs, number_time_intervals))
    
    # create an array of random states - necessary to replicate returned data
    random_states = sc.randint.rvs(0,number_jobs*number_time_intervals,
                                   size=(number_jobs, number_time_intervals),
                                   random_state=random_state)
    
    for row in range(number_jobs):
        time_horizon_profile = get_time_horizon_profiles_by_time_interval(job_time_horizon_profiles_dict[row], 
                                                                          number_time_intervals)
        zero_taskload_chance = job_zero_taskload_probability_dict[row]
        for column in range(number_time_intervals):
            mean = job_normal_dist_params_dict[row]['mean'] * time_horizon_profile[column]
            stdev = job_normal_dist_params_dict[row]['stdev'] * time_horizon_profile[column]
            random_state_n = random_states[row, column]
            if sc.uniform.rvs(0,1,random_state=random_state_n) >= zero_taskload_chance:
                taskload_array[row, column] = np.round(sc.norm.rvs(mean, stdev, size=1, 
                                                                   random_state=random_state_n).clip(min=0),0)[0]
    taskload_array = taskload_array.astype(int)
    
    return taskload_array

def generate_random_taskload_parameters(number_time_intervals, number_jobs, 
                                       job_taskload_parameter_generator_parameters_dict, 
                                       zero_taskload_density=0.25, time_horizon_profile=None, 
                                       random_state=None):

    """
    Function to generate a parameter set, randomised to a certain level given the specified inputs.
    """
    
    # generate taskload parameters for each job
    job_taskload_parameter_generator_mean = job_taskload_parameter_generator_parameters_dict['taskload_dist_means_mean']
    job_taskload_parameter_generator_stdev = job_taskload_parameter_generator_parameters_dict['taskload_dist_means_stdev']
    job_taskload_parameter_generator_stdev_ratio = job_taskload_parameter_generator_parameters_dict['taskload_dist_stdev_ratio']
    job_normal_dist_means = np.round(sc.norm.rvs(job_taskload_parameter_generator_mean, 
                                                  job_taskload_parameter_generator_stdev, 
                                                  size=number_jobs, random_state=random_state).clip(min=0),0)
    job_normal_dist_stdevs = np.round(np.multiply(job_normal_dist_means, 
                                                  job_taskload_parameter_generator_stdev_ratio).clip(min=0),0)
    job_normal_dist_params_dict = {i: {'mean': job_normal_dist_means[i], 'stdev': job_normal_dist_stdevs[i]}
                                   for i in range(number_jobs)}
    
    # if a time_horizon_profile is not provided, just use random ones for each job
    # if it is provided, use that for all jobs
    time_horizon_profiles_dict = return_time_horizon_profiles_dict(random_state=random_state)
    if time_horizon_profile is None:
        time_horizon_profiles_keys = sorted(list(time_horizon_profiles_dict.keys()))
        random_indexes = sc.randint.rvs(0,len(time_horizon_profiles_dict),size=number_jobs,random_state=random_state)
        job_time_horizon_profiles_dict = {i: time_horizon_profiles_keys[random_indexes[i]] for i in range(number_jobs)}
    else:
        job_time_horizon_profiles_dict = {i: time_horizon_profile  for i in range(number_jobs)}
        
    # calculate zero taskload probabilty using the zero_taskload_density
    # parameter as a knob to control density
    job_zero_taskload_probability_dict = {k: (lambda v: 0 if zero_taskload_density == 0 
                                              else 1/pow(v['mean'], 1-zero_taskload_density))(v)
                                          for k, v in job_normal_dist_params_dict.items()}
    
    out_parameters = {
        'number_time_intervals' : number_time_intervals,
        'number_jobs' : number_jobs,
        'job_normal_dist_params_dict' : job_normal_dist_params_dict,
        'job_time_horizon_profiles_dict' : job_time_horizon_profiles_dict,
        'job_zero_taskload_probability_dict' : job_zero_taskload_probability_dict,
        'time_horizon_profiles_dict' : time_horizon_profiles_dict,
    }
    
    return out_parameters
        
def generate_agent_availability_array(number_agents, number_time_intervals, agent_availability_time_intervals_dict):
    
    """
    Function to generate an array of booleans where each row is an agent, each column is
    a time period, and each element indicates whether that agent is working in that
    time period.
    """
    
    out_array = np.zeros((number_agents, number_time_intervals))
    
    for k, v in agent_availability_time_intervals_dict.items():
        
        out_array[k, v[0]:v[1]+1] = np.ones(v[1]-v[0]+1)
        
    return out_array

def return_agent_availabilities(taskload_parameters, number_agents, agent_availability_profile, 
                                shift_length=0, random_state=None):
    
    """
    Function to return an agent availabilties dictionary. Currently the two options are:
        1) full - here each agent can work for the whole time horizon
        2) random - here agents starts are randomly distributed between the first
                    time interval and the last one which would allow them to complete
                    a full shift
    More may be added at a later date.
    """
    
    number_time_intervals = taskload_parameters['number_time_intervals']
    number_jobs = taskload_parameters['number_jobs']
    
    if agent_availability_profile == 'full':
        
        agent_availability_time_intervals_dict = {i: (0, number_time_intervals-1) 
                                                  for i in range(number_agents)}
        
    elif agent_availability_profile == 'random':
        
        start_times = sc.randint.rvs(0, number_time_intervals-shift_length, size=number_agents, 
                                     random_state=random_state)
        
        agent_availability_time_intervals_dict = {i: (start_times[i], start_times[i]+shift_length-1) 
                                                  for i in range(number_agents)}
        
    else:
        agent_availability_time_intervals_dict = {}
        warnings.warn('Please use supported agent availability profile.')
    
    return agent_availability_time_intervals_dict

def generate_agent_job_endorsement_array(number_agents, number_jobs, agent_job_endorsements_dict):
    
    """
    Function to generate an array of booleans where each row is an agent, each column is
    a job, and each element indicates whether that agent is able to work that job.
    """
    
    out_array = np.zeros((number_agents, number_jobs))
    
    for k, v in agent_job_endorsements_dict.items():
        
        out_array[k, v] = 1
        
    return out_array

def return_agent_endorsements(number_agents, number_jobs, endorsement_density_distribution_dict, random_state=None):
    
    """
    Function to return an agent endorsement dictionary. The ratio of jobs that agents can do is 
    sampled from a uniform distribution for each agent, with the upper and lower bounds given in 
    endorsement_density_distribution_dict.
    """
    
    lower_bound = endorsement_density_distribution_dict['lower_bound']
    upper_bound = endorsement_density_distribution_dict['upper_bound']
    
    endorsement_densities = sc.uniform.rvs(scale=upper_bound-lower_bound,loc=lower_bound
                                          ,size=number_agents,random_state=random_state)
    
    uniform_array = sc.uniform.rvs(size=(number_agents, number_jobs), random_state=random_state)
    
    agent_job_endorsements_dict = {i: [j for j in range(number_jobs) 
                                       if uniform_array[i,j] <= endorsement_densities[i]
                                      ]
                                   for i in range(number_agents)
                                  }
    
    return agent_job_endorsements_dict

def generate_random_agent_parameters(number_agents, taskload_parameters, endorsement_density_distribution_dict,
                                     agent_availability_profile, shift_length=0, random_state=None):
    
    """
    Function to generate a random set of agent parameters.
    """

    number_time_intervals = taskload_parameters['number_time_intervals']
    number_jobs = taskload_parameters['number_jobs']
    
    agent_job_endorsements_dict = return_agent_endorsements(number_agents, number_jobs, 
                                                            endorsement_density_distribution_dict, 
                                                            random_state=random_state)
    
    agent_availability_time_intervals_dict = return_agent_availabilities(taskload_parameters, number_agents, 
                                                                         agent_availability_profile, 
                                                                         shift_length=shift_length, 
                                                                         random_state=None)
    
    agent_parameters = {
        'number_agents' : number_agents,
        'agent_availability_time_intervals_dict' : agent_availability_time_intervals_dict,
        'agent_job_endorsements_dict' : agent_job_endorsements_dict
    }
    
    return agent_parameters

def create_taskload_heatmap(taskload_parameters, taskload_array, in_notebook=False,
                            colorscale='Jet',annotate_heatmap=True):
    
    """
    Create a heatmap to visualise the taskload array.
    """
    
    number_time_intervals = taskload_parameters['number_time_intervals']
    number_jobs = taskload_parameters['number_jobs']
    
    time_interval_taskloads = taskload_array.sum(axis=0)
    
    
    z_hover = np.empty((number_jobs, number_time_intervals),dtype='object')
    for i in range(number_time_intervals):
        for j in range(number_jobs):
            z_hover[j, i] = 'time interval ' + str(i) + '  sector #' + str(j) \
            + '   taskload=' + str(taskload_array[j, i])
            
    if annotate_heatmap:
        annotation_text = taskload_array
    else:
        annotation_text = np.empty(taskload_array.shape,dtype='str')
        
    trace = ff.create_annotated_heatmap(z=taskload_array
    , x=[i for i in range(number_time_intervals)]
    , y=[i for i in range(number_jobs)]
    , xgap=1, ygap=1
    , annotation_text=annotation_text
    , colorscale=colorscale
    , text=z_hover
    , hoverinfo='text'
    )
    
    trace.layout.update({'title':'Taskload Matrix',
                        'xaxis':go.layout.XAxis(#title='Time Intervals',
                        tickvals = [i for i in range(number_time_intervals)],
                        ticktext = [str(int(i)) for i in time_interval_taskloads],
                        tickfont = {'size':10},
                        tickangle=30,
                        side='top'),                  
                                                
                         
                         'yaxis':go.layout.YAxis(title='Job ID')})
    
    if in_notebook:
        iplot(trace)
    else:
        plot(trace)
        
        
def is_solution_valid(taskload_array, solution, partition_max):

    max_partition_index = int(solution.max())
    
    valid = True
    for ti in range(solution.shape[1]): 
        
        for partition in range(max_partition_index+1):
            
            job_indexes = np.where(solution[:,ti]==partition)[0]
            temp_array = np.zeros(solution.shape[0])
            temp_array[job_indexes] = 1
            job_workload = np.multiply(temp_array, taskload_array[:,ti]).sum()
                                                 
            if job_workload > partition_max:
                warnings.warn('solution not valid for partition ' + str(partition) + ' in time interval ' + str(ti))
                valid = False
        
    return valid

def make_solution_groups_array(model, taskload_parameters):

    number_jobs = taskload_parameters['number_jobs']
    number_time_intervals = taskload_parameters['number_time_intervals']
    
    solution_array = np.zeros((number_jobs, number_time_intervals))
    
    job_time_to_group_dict = {(k2[0], k2[2]): k2[1] for k2, v2 in 
                                    {k: v for k, v in model.p.get_values().items() if v == 1
                                }.items()}
    
    for k, v in job_time_to_group_dict.items():
        
        solution_array[k[0],k[1]] = v
        
    return solution_array


def make_solution_agents_array(model, taskload_parameters):
    
    number_jobs = taskload_parameters['number_jobs']
    number_time_intervals = taskload_parameters['number_time_intervals']
    
    solution_array = np.zeros((number_jobs, number_time_intervals))
    
    job_time_to_group_dict = {(k2[0], k2[2]): k2[1] for k2, v2 in 
                                    {k: v for k, v in model.p.get_values().items() if v == 1
                                }.items()}
    group_time_to_agent_dict = {(k2[1], k2[2]): k2[0] for k2, v2 in 
                                        {k: v for k, v in model.q.get_values().items() if v == 1
                                    }.items()}
    job_time_to_agent_dict = {k: group_time_to_agent_dict[(v, k[1])]  for k, v in job_time_to_group_dict.items()}
    
    for k, v in job_time_to_agent_dict.items():
        
        solution_array[k[0],k[1]] = v
        
    return solution_array


def make_solution_heatmap(model, taskload_parameters, agent_parameters, taskload_array, 
                          agent_heatmap = True, in_notebook=False, colorscale='Jet',
                          annotate_heatmap=True):
    
    number_jobs = taskload_parameters['number_jobs']
    number_time_intervals = taskload_parameters['number_time_intervals']
    
    if agent_heatmap:
        solution_array = make_solution_agents_array(model, taskload_parameters)
    else:
        solution_array = make_solution_groups_array(model, taskload_parameters)
    
    time_interval_taskloads = taskload_array.sum(axis=0)
    
    z_hover = np.empty((number_jobs, number_time_intervals),dtype='object')
    for i in range(number_time_intervals):
        for j in range(number_jobs):
            z_hover[j, i] = 'time interval ' + str(i) + '  job #' + str(j)
            
    if annotate_heatmap:
        annotation_text = taskload_array
    else:
        annotation_text = np.empty(taskload_array.shape,dtype='str')
        
    trace = ff.create_annotated_heatmap(z=solution_array
        , x=[i for i in range(number_time_intervals)]
        , y=[i for i in range(number_jobs)]
        , xgap=1, ygap=1
        , annotation_text=annotation_text
        , colorscale=colorscale
        , text=z_hover
        , hoverinfo='text'
        , showscale=True
    )
    
    trace.layout.update({'title':'Job to Console Solution',
                        'xaxis':go.layout.XAxis(#title='Time Intervals',
                        tickvals = [i for i in range(number_time_intervals)],
                        ticktext = [str(int(i)) for i in time_interval_taskloads],
                        tickfont = {'size':10},
                        tickangle=30,
                        side='top'),                  
                                                
                         
                         'yaxis':go.layout.YAxis(title='Job ID')})
    
    if in_notebook:
        iplot(trace)
    else:
        plot(trace)
        
        

def get_solution_stats(group_solution_array, agent_solution_array):
    
    solution_stats = {}
    
    # helper function to get number of unique elements in each column of an array
    def nunique_percol_sort(a):
        b = np.sort(a,axis=0)
        return (b[1:] != b[:-1]).sum(axis=0)+1
    
    solution_stats['time_intervals_working'] = nunique_percol_sort(group_solution_array).sum()
    
    group_job_reconfig_count = 0
    group_reconfig_count = 0
    for i in range(group_solution_array.shape[0]):
        for j in range(group_solution_array.shape[1]-1):
            if i == 0:
                if not np.array_equal(group_solution_array[:,j], group_solution_array[:,j+1]):
                    group_reconfig_count += 1
            if group_solution_array[i,j] != group_solution_array[i,j+1]:
                group_job_reconfig_count += 1
    
    solution_stats['group_job_reconfig_count'] = group_job_reconfig_count
    solution_stats['group_reconfig_count'] = group_reconfig_count
    
    agent_reconfig_count = 0
    for j in range(agent_solution_array.shape[1]-1):
        if not np.array_equal(agent_solution_array[:,j], agent_solution_array[:,j+1]):
            agent_reconfig_count += min(len(np.unique(agent_solution_array[:,j])), 
                                        len(np.unique(agent_solution_array[:,j+1])))
            
    solution_stats['agent_reconfig_count'] = agent_reconfig_count
    
    return solution_stats


### Generate a dummy problem

In this section we will generate some data for a sample problem, using functions from a previous notebook.

We first generate some taskload parameters. Here we are using: 

- 56 time intervals

- 14 sectors

- Seector taskload (Normal) distribution means generated by N~(100, 10), with their standard deviations a quarter of the mean

- A low level of zero valued taskloads

- Taskloads distributed to be higher at the beginning and end of the time horizon

In [8]:
num_time_intervals = 56
num_sectors = 14

taskload_parameters = generate_random_taskload_parameters(num_time_intervals, num_sectors, 
                                                          {'taskload_dist_means_mean' : 100, 
                                                           'taskload_dist_means_stdev' : 10,
                                                           'taskload_dist_stdev_ratio' : 0.25},
                                                          zero_taskload_density=0.25, 
                                                          time_horizon_profile='middle_heavy', 
                                                          random_state=1)
taskload_parameters

{'number_time_intervals': 56,
 'number_jobs': 14,
 'job_normal_dist_params_dict': {0: {'mean': 116.0, 'stdev': 29.0},
  1: {'mean': 94.0, 'stdev': 24.0},
  2: {'mean': 95.0, 'stdev': 24.0},
  3: {'mean': 89.0, 'stdev': 22.0},
  4: {'mean': 109.0, 'stdev': 27.0},
  5: {'mean': 77.0, 'stdev': 19.0},
  6: {'mean': 117.0, 'stdev': 29.0},
  7: {'mean': 92.0, 'stdev': 23.0},
  8: {'mean': 103.0, 'stdev': 26.0},
  9: {'mean': 98.0, 'stdev': 24.0},
  10: {'mean': 115.0, 'stdev': 29.0},
  11: {'mean': 79.0, 'stdev': 20.0},
  12: {'mean': 97.0, 'stdev': 24.0},
  13: {'mean': 96.0, 'stdev': 24.0}},
 'job_time_horizon_profiles_dict': {0: 'middle_heavy',
  1: 'middle_heavy',
  2: 'middle_heavy',
  3: 'middle_heavy',
  4: 'middle_heavy',
  5: 'middle_heavy',
  6: 'middle_heavy',
  7: 'middle_heavy',
  8: 'middle_heavy',
  9: 'middle_heavy',
  10: 'middle_heavy',
  11: 'middle_heavy',
  12: 'middle_heavy',
  13: 'middle_heavy'},
 'job_zero_taskload_probability_dict': {0: 0.028291534783718025,
  1: 0.

In [16]:
taskload_array = generate_taskload_array(taskload_parameters, random_state=0)
create_taskload_heatmap(taskload_parameters, taskload_array, in_notebook=True, 
                        colorscale='Greens',annotate_heatmap=False)

We need to generate some aggregate sectors made up of elementary sectors.

In [ ]:
def generate_aggregate_sectors(num_elementary_sectors, min_aggregate_size, max_aggregate_size):
    
    # put in one sector which covers the whole airspace to start
    aggregate_sector_dict = {0: [i for i in range(num_elementary_sectors)]}
    
    for 

We also need to generate some valid configurations that the airspaces can take.

In [75]:
def peturb_config_airspaces(config, peturb_index, peturb_direction):
    
    """
    Function to peturb an existing configuration of airspaces. Takes the airspace 
    (list of airspace indexes) at peturb index and moves one airspace to either the
    airspace on the right or left, as specified by the input. If the peturb index 
    is 0 then the airspace has to move to the right, and likewise if the index is
    the last one, the move has to be to the right. If the airspace at peturb index
    only has one elementary airspace (index) in it then we don't peturb anything 
    and just return the original config.
    """
    
    if peturb_direction not in ['left','right']:
        raise ValueError('The peturb direction can only be left or right')
    
    if peturb_index == 0:
        peturb_direction = 'right'
    if peturb_index == len(config) - 1:
        peturb_direction = 'left'
    
    # only peturb the config if the airspace being altered has more than one 
    # elementary sector in it
    if len(config[peturb_index]) > 1:
        if peturb_direction == 'right':
            new_config = config[:peturb_index] + [config[peturb_index][:-1]] + \
                [[config[peturb_index][-1]] + config[peturb_index+1]] + config[peturb_index+2:]
        if peturb_direction == 'left':
            new_config = config[:peturb_index-1] + [config[peturb_index-1] + \
                [config[peturb_index][0]]] + [config[peturb_index][1:]] + config[peturb_index+1:]
        return new_config
    else:
        return config

In [81]:
peturb_config_airspaces([list(l) for l in np.array_split(np.arange(10), 4)], 2, 'right')

[[0, 1, 2], [3, 4, 5], [6], [7, 8, 9]]

In [ ]:
def generate_airspaces_and_configurations(num_elementary_sectors, max_groups, num_configs_per_group_size):
    
    """
    Function to generate a set of configurations and the airspaces populating those configurations.
    Will use the config peturb function to give different but reasonably balanced configs, from
    which all the distinct airspaces will be taken.
    """
    
    elementary_sector_array = np.arange(num_elementary_sectors)
       
    # put in one sector which covers the whole airspace to start, as the first configuration
    airspaces_dict = {0: elementary_sector_array}
    configuration_dict = {0: np.array([0])}
    
    counter = 1
    for num_groups in range(2, max_groups+1):
        first_config = np.array_split(ar, num_groups)
        first_config = [list(l) for l in first_config]
        for group_index in range(num_groups):
            if len(first_config[group_index]) > 1:
                 if group_index == 0:
                    config = [first_config[0][:-1]] + [[first_config[1][-1]] + first_config[0][:-1]

In [41]:
ar = np.arange(10)
splt = np.array_split(ar, 4)
splt

[array([0, 1, 2]), array([3, 4, 5]), array([6, 7]), array([8, 9])]

In [43]:
splt[0][-1]

2

Write a model - at first ignore configurations and work solely with airspaces

In [ ]:
def create_airspace_variables(num_sectors, num_time_intervals):
    
    airspace_variables = {}
    for sector in range(num_sectors):
        for ti in range(num_time_intervals):

In [ ]:
def create_model():
    
    # create model
    model = cp_model.CpModel()